First:
- go to the terminal
- go to the `4_tuning_networks` folder, use `cd` to get there, or `z` to get there faster.
- if you are planning on closing the laptop to let this run by itself, start a tmux session by giving the command `tmux`. You can name the session with `ctrl+b $`, leave the tmux with `ctrl+b` d, access it again with `tmux a -t 0` or replace the 0 with the name you gave it. See [tmux](https://github.com/tmux/tmux/wiki/Getting-Started) for more info.
- Activate the `.venv`, and run `python hypertune.py`
- In the hypertune.py file, a tune_dir is specified: `models/ray`. We will check the contents of that folder after the hypertune finished. You can also use tensorboard to check the results. 
- this will take some time. I clocked 1901 seconds (30 minutes) on the VM

In [ ]:
from pathlib import Path
tune_dir = Path("models/ray").resolve()
tune_dir.exists()

In [ ]:
tunelogs = [d for d in tune_dir.iterdir()]
tunelogs.sort()
latest = tunelogs[-1]
latest

In [ ]:
from ray.tune import ExperimentAnalysis
import ray
ray.init(ignore_reinit_error=True)


analysis = ExperimentAnalysis(latest)


So, we find some info:

In [ ]:
analysis.results_df.columns

Print first rows

In [ ]:
plot = analysis.results_df

columns_of_interest = ['iterations', 'train_loss', 'test_loss', 'Accuracy',
                     'training_iteration','time_this_iter_s', 'time_total_s',
                       'config/input_size', 'config/output_size', 'config/hidden_size', 'config/dropout',
                       'config/num_layers', 'config/optimizer']

result_table = plot.loc[:,columns_of_interest]
result_table.sort_values("Accuracy", ascending=False, inplace=True)
result_table

Let's focus on the parameters we wanted to tune.

Kolommen volgorde aangepast zodat accuracy rechts staat. Leest makkelijker

In [ ]:
import plotly.express as px

# Zorg ervoor dat 'analysis' en 'results_df' correct zijn gedefinieerd
plot = analysis.results_df
select = ["config/optimizer", "config/hidden_size", "config/num_layers", "config/dropout", "Accuracy"]
p = plot[select].reset_index().dropna()

# Maak parallelle coördinaten plots met het gespecificeerde asbereik
fig_adam = px.parallel_coordinates(p[p["config/optimizer"] == "Adam"], color="Accuracy", title="Parallel Coordinates for Adam Optimizer", dimensions=["config/hidden_size", "config/num_layers", "config/dropout", "Accuracy"])
fig_sgd = px.parallel_coordinates(p[p["config/optimizer"] == "SGD"], color="Accuracy", title="Parallel Coordinates for SGD Optimizer", dimensions=["config/hidden_size", "config/num_layers", "config/dropout", "Accuracy"])	

fig_adam.show()
fig_sgd.show()


Let's sort by accuracy

In [ ]:
p.sort_values("Accuracy", inplace=True)

Make a parallel plot

In [ ]:
px.parallel_coordinates(p, color="Accuracy")

In [ ]:
import seaborn as sns

sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter data for Adam optimizer
p_adam = p[p["config/optimizer"] == "Adam"]
# Filter data for SGD optimizer
p_sgd = p[p["config/optimizer"] == "SGD"]

# Create scatter plot for Adam optimizer
plt.figure(figsize=(8, 4))
sns.scatterplot(data=p_adam, x="config/hidden_size", y="config/dropout", hue="Accuracy", palette="coolwarm")
plt.title("Scatter Plot for Adam Optimizer")
plt.show()

# Create scatter plot for SGD optimizer
plt.figure(figsize=(8, 4))
sns.scatterplot(data=p_sgd, x="config/hidden_size", y="config/dropout", hue="Accuracy", palette="coolwarm")
plt.title("Scatter Plot for SGD Optimizer")
plt.show()

In [ ]:
import seaborn as sns

pivot_table = p.pivot_table(index="config/hidden_size", columns="config/num_layers", values="Accuracy", aggfunc='mean').sort_index(ascending=False)
#sns.heatmap(pivot_table, cmap="coolwarm", annot=True)
# Separate pivot tables for Adam and SGD optimizers
pivot_table_adam = p[p["config/optimizer"] == "Adam"].pivot_table(index="config/hidden_size", columns="config/num_layers", values="Accuracy", aggfunc='mean').sort_index(ascending=False)
pivot_table_sgd = p[p["config/optimizer"] == "SGD"].pivot_table(index="config/hidden_size", columns="config/num_layers", values="Accuracy", aggfunc='mean').sort_index(ascending=False)

# Plot heatmap for Adam optimizer
plt.figure(figsize=(8,4))
sns.heatmap(pivot_table_adam, cmap="coolwarm", annot=True)
plt.title("Heatmap of Accuracy for Adam Optimizer")
plt.show()

# Plot heatmap for SGD optimizer
plt.figure(figsize=(8,4))
sns.heatmap(pivot_table_sgd, cmap="coolwarm", annot=True)
plt.title("Heatmap of Accuracy for SGD Optimizer")
plt.show()

In [ ]:
import seaborn as sns

pivot_table = p.pivot_table(index="config/hidden_size", columns="config/num_layers", values="Accuracy", aggfunc='mean').sort_index(ascending=False)
#sns.heatmap(pivot_table, cmap="coolwarm", annot=True)
# Separate pivot tables for Adam and SGD optimizers
pivot_table_adam = p[p["config/optimizer"] == "Adam"].pivot_table(index="config/hidden_size", columns="config/dropout", values="Accuracy", aggfunc='mean').sort_index(ascending=False)
pivot_table_sgd = p[p["config/optimizer"] == "SGD"].pivot_table(index="config/hidden_size", columns="config/dropout", values="Accuracy", aggfunc='mean').sort_index(ascending=False)

# Plot heatmap for Adam optimizer
plt.figure(figsize=(8,4))
sns.heatmap(pivot_table_adam, cmap="coolwarm", annot=True)
plt.title("Heatmap of Accuracy for Adam Optimizer")
plt.show()

# Plot heatmap for SGD optimizer
plt.figure(figsize=(8,4))
sns.heatmap(pivot_table_sgd, cmap="coolwarm", annot=True)
plt.title("Heatmap of Accuracy for SGD Optimizer")
plt.show()

In [ ]:
import seaborn as sns

p["config/dropout"] = p["config/dropout"].round(4)
pivot_table = p.pivot_table(index="config/hidden_size", columns="config/dropout", values="Accuracy", aggfunc='mean').sort_index(ascending=False)
sns.heatmap(pivot_table, cmap="coolwarm", annot=True)

In [ ]:
import matplotlib.pyplot as plt
cmap = sns.cubehelix_palette(as_cmap=True)
sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")
sns.kdeplot(data=p, x="config/hidden_size", y="config/num_layers", cmap=cmap)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

Get the best trial

In [ ]:
analysis.get_best_trial(metric="test_loss", mode="min")

The top ten

In [ ]:
p[-10:]

Or the best config

In [ ]:
analysis.get_best_config(metric="Accuracy", mode="max")